In [1]:
import pandas as pd
from utils.preprocess import *
from utils.model import *
from tqdm import tqdm

In [2]:
def cat_preprocess(df, drop_cols, encode_col):
    df = df.drop(drop_cols, axis = 1)
    df = pd.get_dummies(df, columns=[encode_col])
    return df

In [3]:
def cat_feat_extr(df, cols, ves_col):
    for NPLV in tqdm(df.NPLV.unique()):
        for col in cols:
            if not df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()].empty: 
                for i in range(len(df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()])):
                    df.at[df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i], col] = \
                            df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i]]
                    
    df = df.drop([ves_col], axis = 1).groupby('NPLV').agg('sum')
    
    return df

In [4]:
lom_train = pd.read_csv(config['data_path'] + 'lom_train.csv')
sip_train = pd.read_csv(config['data_path'] + 'sip_train.csv')
lom_test = pd.read_csv(config['data_path'] + 'lom_test.csv')
sip_test = pd.read_csv(config['data_path'] + 'sip_test.csv')

In [5]:
prep_lom_train = cat_preprocess(lom_train, 'VDL', 'NML')
prep_sip_train = cat_preprocess(sip_train, ['VDSYP','DAT_OTD'], 'NMSYP')
prep_lom_test = cat_preprocess(lom_test, 'VDL', 'NML')
prep_sip_test = cat_preprocess(sip_test, ['VDSYP','DAT_OTD'], 'NMSYP')

In [6]:
prep_lom_train_cols = list(prep_lom_train.columns.drop(['NPLV', 'VES']))
prep_sip_train_cols = list(prep_sip_train.columns.drop(['NPLV', 'VSSYP']))
prep_lom_test_cols = list(prep_lom_test.columns.drop(['NPLV', 'VES']))
prep_sip_test_cols = list(prep_sip_test.columns.drop(['NPLV', 'VSSYP']))

In [7]:
prep_lom_train[prep_lom_train_cols] = prep_lom_train[prep_lom_train_cols].astype('int64')
prep_sip_train[prep_sip_train_cols] = prep_sip_train[prep_sip_train_cols].astype('int64')
prep_lom_test[prep_lom_test_cols] = prep_lom_test[prep_lom_test_cols].astype('int64')
prep_sip_test[prep_sip_test_cols] = prep_sip_test[prep_sip_test_cols].astype('int64')

In [8]:
cat_lom_train = cat_feat_extr(prep_lom_train, prep_lom_train_cols, 'VES')
cat_sip_train = cat_feat_extr(prep_sip_train, prep_sip_train_cols, 'VSSYP')
cat_lom_test = cat_feat_extr(prep_lom_test, prep_lom_test_cols, 'VES')
cat_sip_test = cat_feat_extr(prep_sip_test, prep_sip_test_cols, 'VSSYP')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 780/780 [00:09<00:00, 82.45it/s]


In [9]:
cat_lom_train = cat_lom_train.reset_index()
cat_sip_train = cat_sip_train.reset_index()
cat_lom_test = cat_lom_test.reset_index()
cat_sip_test = cat_sip_test.reset_index()

In [10]:
drop_lom_train = list(set(cat_lom_train.columns).difference(set(cat_lom_test.columns)))
drop_sip_train = list(set(cat_sip_train.columns).difference(set(cat_sip_test.columns)))
drop_lom_test = list(set(cat_lom_test.columns).difference(set(cat_lom_train.columns)))
drop_sip_test = list(set(cat_sip_test.columns).difference(set(cat_sip_train.columns)))

In [11]:
cat_lom_train = cat_lom_train.drop(drop_lom_train, axis=1)
cat_lom_test = cat_lom_test.drop(drop_lom_test, axis=1)
cat_sip_train = cat_sip_train.drop(drop_sip_train, axis=1)
cat_sip_test = cat_sip_test.drop(drop_sip_test, axis=1)

In [12]:
cat_lom_train.to_csv(config['data_path'] + 'cat_lom_train.csv', index=False)
cat_sip_train.to_csv(config['data_path'] + 'cat_sip_train.csv', index=False)
cat_lom_test.to_csv(config['data_path'] + 'cat_lom_test.csv', index=False)
cat_sip_test.to_csv(config['data_path'] + 'cat_sip_test.csv', index=False)